<a href="https://colab.research.google.com/github/siddharth-kale/DNA3/blob/main/DNA3_Aircraft_Inventory_Flight_Status.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade numpy
!pip install --upgrade traffic

     |████████████████████████████████| 15.7 MB 119 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.4 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 29.6 MB 125 kB/s 
     |████████████████████████████████| 62 kB 746 kB/s 
     |████████████████████████████████| 12.1 MB 45.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 2.7 MB 31.0 MB/s 
     |████████████████████████████████| 6.3 MB 41.2 MB/s 
     |████████████████████████████████| 11.2 MB 24.1 MB/s 
     |████████████████████████████████| 58 kB 4.8 MB/s 
     |████████████████████████████████| 211 kB 42.1 MB/s 
     |████████████████████████████████| 206 kB 47.9 MB/s 
     |████████████████████████████████| 3.3 MB 50.3 MB/s 
     |████████████████████████████████| 6.3 MB 35.3 MB/s 
     |████████████████████████████████| 38.2 MB 1.1 MB/s 
     |████████████████████████████████| 97 kB 6.2 MB/s 
     |████████████████████████████████| 11.3 MB 19.7 MB/s 
     |███

In [1]:
import pandas as pd
import numpy as np
import seaborn as sea
import matplotlib.pyplot as plt
from google.colab import drive
import glob
import plotly.express as px

In [2]:
import traffic
from traffic.data import opensky
from traffic.core import Traffic

In [3]:
pd.options.display.max_rows=999

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
files = glob.glob('/content/gdrive/MyDrive/Projects/DNA3/Data/dna3inventory/*.csv')

raw_df = pd.DataFrame()

for f in files:
  temp_df = pd.read_csv(f).drop(columns='Unnamed: 0')
  raw_df = raw_df.append(temp_df)

raw_df

,alert,altitude,callsign,geoaltitude,groundspeed,hour,icao24,last_position,latitude,longitude,onground,spi,squawk,timestamp,track,vertical_rate
0,False,NaN,NaN,NaN,387.0,2019-11-29 01:00:00+00:00,89618a,NaN,NaN,NaN,False,False,NaN,2019-11-29 01:28:37+00:00,300.277764,-1472.0
1,False,NaN,NaN,NaN,387.0,2019-11-29 01:00:00+00:00,89618a,NaN,NaN,NaN,False,False,NaN,2019-11-29 01:28:38+00:00,300.277764,-1472.0
2,False,NaN,NaN,NaN,387.0,2019-11-29 01:00:00+00:00,89618a,NaN,NaN,NaN,False,False,NaN,2019-11-29 01:28:39+00:00,300.277764,-1472.0
3,False,NaN,NaN,NaN,387.0,2019-11-29 01:00:00+00:00,89618a,NaN,NaN,NaN,False,False,NaN,2019-11-29 01:28:40+00:00,300.277764,-1472.0
4,False,NaN,NaN,NaN,387.0,2019-11-29 01:00:00+00:00,89618a,NaN,NaN,NaN,False,False,NaN,2019-11-29 01:28:41+00:00,300.277764,-1472.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213308,False,NaN,UAE924,NaN,NaN,2021-01-15 22:00:00+00:00,89618c,2021-01-15 22:49:42.632999936+00:00,25.259342,55.345904,True,False,NaN,2021-01-15 22:54:38+00:00,NaN,NaN
1213309,False,NaN,UAE924,NaN,NaN,2021-01-15 22:00:00+00:00,89618c,2021-01-15 22:49:42.632999936+00:00,25.259342,55.345904,True,False,NaN,2021-01-15 22:54:39+00:00,NaN,NaN
1213310,False,NaN,UAE924,NaN,NaN,2021-01-15 22:00:00+00:00,89618c,2021-01-15 22:49:42.632999936+00:00,25.259342,55.345904,True,False,NaN,2021-01-15 22:54:40+00:00,NaN,NaN
1213311,False,NaN,UAE924,NaN,NaN,2021-01-15 22:00:00+00:00,89618c,2021-01-15 22:49:42.632999936+00:00,25.259342,55.345904,True,False,NaN,2021-01-15 22:54:41+00:00,NaN,NaN


In [7]:
raw_df['timestamp'] = raw_df['timestamp'].astype(dtype='datetime64')
raw_df['date'] = raw_df['timestamp'].apply(lambda x:x.date())
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1213313 entries, 0 to 1213312
Data columns (total 17 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   alert          1213313 non-null  object        
 1   altitude       1001694 non-null  float64       
 2   callsign       1179005 non-null  object        
 3   geoaltitude    893309 non-null   float64       
 4   groundspeed    971912 non-null   float64       
 5   hour           1213313 non-null  object        
 6   icao24         1213313 non-null  object        
 7   last_position  1121831 non-null  object        
 8   latitude       1121831 non-null  float64       
 9   longitude      1121831 non-null  float64       
 10  onground       1213313 non-null  object        
 11  spi            1213313 non-null  object        
 12  squawk         1064842 non-null  float64       
 13  timestamp      1213313 non-null  datetime64[ns]
 14  track          971912 non-null   f

In [8]:
fleet_df = raw_df.groupby(by=['date','icao24']).agg(
    {
        'groundspeed':['mean'],
     
    }
).reset_index().fillna(0)

fleet_df.columns=['date','ICAO','groundspeed']

fleet_df['airborne'] = fleet_df['groundspeed'].apply(lambda x: 1 if x>0 else 0)
fleet_df

,date,ICAO,groundspeed,airborne
0,2019-11-29,89618a,419.018507,1
1,2019-11-29,89618c,273.818599,1
2,2019-11-29,89618e,502.156140,1
3,2019-12-06,89618a,219.758722,1
4,2019-12-06,89618c,355.523370,1
5,2019-12-06,89618e,452.373925,1
6,2019-12-13,89618a,215.973437,1
7,2019-12-13,89618b,0.000000,0
8,2019-12-13,89618c,325.096270,1
9,2019-12-13,89618e,463.006809,1


In [9]:
final_df = fleet_df.groupby(by='date').agg(
    {
        'airborne':['sum']
    }
).reset_index()

final_df

,date,airborne
,,sum
0,2019-11-29,3
1,2019-12-06,3
2,2019-12-13,3
3,2019-12-20,4
4,2019-12-27,4
5,2020-01-03,4
6,2020-01-10,4
7,2020-01-17,4
8,2020-01-24,4


In [10]:
fig=px.bar(final_df,x='date',y='airborne',text='airborne')
fig.update_layout(plot_bgcolor='white')
fig.update_traces(textposition='outside',marker_color='rgb(10,10,105)',)
fig.show()